I am really enjoying having this weather station.  I say weather
station, but it is just a raspberry pi with a pressure and temperature
sensor attached to it.
Computers are versatile, you can run any software on them, so they can
do a lot of different things.

But the pi takes this to a whole different level.  They are $40 or so,
depending on taxes and stuff.  Now you need a whole bunch of other
stuff: leads, keyboards, sensors, cameras, touch screens and lots more.

I have been using [Adafruit]().  They have been very helpful with orders.

I found browsing for parts and finding what I needed to get started
with the weather stuff a bit confusing for a while.

I had other stuff to do anyway, I needed to get used to just
installing software on them and setting up environments where I can
figure out how things are working.

I found a great [posting]() about building a weather station, with
enough detail I thought I would be able to work through it.

I order a cheap humidity sensor ($5) that also should do temperature.
I haven't got it working yet, not sure if it is hardware or software.
Meanwhile, I now have a better sensor.  The humidity here is often
outside the range the cheaper sensor is supposed to work.

The good thing is that I should be able to find a use for it at some point.

I also have a camera for this thing and a touchscreen.  I am thinking
of trying to combine them to make a camera.

The one on my phone can do some neat stuff, but the interface keeps
changing and not in ways that are making things easier.

The night skies have been spectacular of late, with Venus and Jupiter
close together in the early evening sky to the west.  If you see two
bright stars after sunset, that is probably them.

Then the moon is just past full.  Here in Bermuda there is often
cloud, not heavy, but patchy clouds.  The humidity is often very high,
so the atmosphere is interesting.

The best sunsets usually have some clouds for the setting sun to
reflect off.  The same with sunrises.

And the full moon too.  Tonight it rose behind cloud.  Last night it
was clearer and it rose orange/red.



### Back to weather
Since I got this thing working it has been hot and settled weather.
There is a strong Bermuda high in place.  Someone described it to me
as like a pit-bull, once it gets hold it does not let go.  So, we may
be in for a long hot spell.

With the pressure changing quite smoothly, the plots I created still
showed quite an interesting pattern.

Skip the next bit, or go back to the earlier [post](), it is just
setting things up to do some plotting.


In [23]:
# Tell matplotlib to plot in line
%matplotlib inline

# import pandas
import pandas

# seaborn magically adds a layer of goodness on top of Matplotlib
# mostly this is just changing matplotlib defaults, but it does also
# provide some higher level plotting methods.
import seaborn

# Tell seaborn to set things up
seaborn.set()



def smooth(data, thresh=None):
    
    means = data.mean()

    if thresh is None:
        sds = data.std()
    else:
        sds = thresh
    
    delta = data - data.shift()
    
    good = delta[abs(delta) < sds]

    print(good.describe())
    
    return delta.where(good, 0.0)

In [24]:
# set the path to the file we are going to analyse
infile = "../files/light.csv"

!scp 192.168.0.133:Adafruit_Python_BMP/light.csv ../files

light.csv                                     100% 1625KB   1.6MB/s   00:00    


light.csv                                       0%    0     0.0KB/s   --:-- ETA

In [25]:

""" assume it is csv and let pandas do magic

  index_col tells it to use the 'date' column in the data
  as the row index, plotting picks up on this and uses the
  date on the x-axis

  The *parse_dates* bit just tells it to try and figure out
  the date/time in the columne labeled 'date'.
"""
data = pandas.read_csv(infile, index_col='date', parse_dates=['date'])

In [26]:
# incantation to extract the first record in the data
start = data[['temp', 'altitude']].irow(0)

# smooth the data to filter out bad temps and pressures
sdata = (smooth(data, 5.0).cumsum() + start)

# now use smooth to throw away dodgy data, and plot the temp and altitude fieldsa
sdata[['temp', 'altitude']].plot(subplots=True)

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f5f6c53f7f0>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x7f5f6c2cef98>], dtype=object)

               temp      pressure      altitude  sealevel_pressure
count  24297.000000  12920.000000  24073.000000       13027.000000
mean       0.000004      0.016409      0.000661          -0.000230
std        0.032970      2.459936      0.515331           2.456391
min       -0.100000     -4.000000     -2.330888          -4.000000
25%        0.000000     -2.000000     -0.331813          -2.000000
50%        0.000000      0.000000      0.000000           0.000000
75%        0.000000      2.000000      0.331889           2.000000
max        0.100000      4.000000      2.748358           4.000000


### Temperature
The temperature plot shows a steady up and down, warming during the
days, cooling a little, but only 2C at night.

There is one day, where I think we had some thunder storms where it dropped during the day.

The last week or so the temperature has been steadily climbing.

### Pressure

The Pressure also shows slow drifting up and down.  But there is this
other strange ripple up and down.

I mentioned this to a meteorologist and immediately got the reply that
it was because the atmosphere is tidal.

So the pattern we see in the pressure should be driven largely by the moon.



In [27]:
import astropy

In [ ]:
from astropy import units
from astropy import find_api_page



In [ ]:
# find_api_page is handy, even opens a browser windo.

#find_api_page(units)

In [ ]:
# astropy is cool, but I need data for the moon.  Let's try pyephem

# uncommented the line below and run this cell if you need to install using
# pip.  This will install into the environment that is being used to run your
# ipython notebook server.

#!pip install pyephem

In [28]:
import ephem

In [ ]:
# Create a Moon

moon = ephem.Moon()

In [ ]:
# Tell it to figure out where it is
moon.compute()

# pring out the phase
moon.phase

In [29]:
def moon_orbitals(index):
    """ Given an index of times create a DataFrame of moon orbitals
    
    For now, just Phase, geocentric latitude and geocentric longitude
    """
    
    # Create dataframe with index as the index
    df = pandas.DataFrame(index=index)
    
    # Add three series
    df['phase'] = pandas.Series()
    df['glat'] = pandas.Series()
    df['glon'] = pandas.Series()
    
    # Now generate the data
    # NB this is slow, solpy might work out faster
    moon = ephem.Moon()
    for ix, timestamp in enumerate(index):
        
        # Compute the moon posigion
        moon.compute(timestamp.strftime("%Y/%m/%d %H:%M:%S"))
        
        df.phase[ix] = moon.phase
        df.glat[ix] = moon.hlat
        df.glon[ix] = moon.hlon
        
    return df
        
        

In [30]:
# See what we got

moon = moon_orbitals(data.index)
moon.describe()

              phase          glat          glon
count  24390.000000  24390.000000  24390.000000
mean      71.811609      0.030610      3.590249
std       25.825346      0.051875      2.041656
min       12.998277     -0.078916      0.000126
25%       55.731966     -0.007763      1.029049
50%       79.226440      0.045898      4.204486
75%       94.245584      0.076175      5.220167
max       99.811104      0.087505      6.283139

In [31]:
moon.plot(subplots=True)

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f5f6c3bf4e0>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x7f5f6c319c18>], dtype=object)

In [9]:
# Try feeding in a longer time series
days = pandas.date_range('7/7/2015', periods=560, freq='D')

In [10]:
moon_orbitals(days).plot(subplots=True)

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f5f6e3f6a58>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x7f5f6e96d978>], dtype=object)

In [32]:
sdata['moon_phase'] = moon.phase
sdata['moon_glat'] = moon.glat
sdata['moon_glon'] = moon.glon

# FIXME -- must be a pandas one liner eg data += moon ?

In [33]:
sdata[['temp', 'altitude', 'moon_phase', 'moon_glon', 'moon_glat']].plot(subplots=True)

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f5f679f7f60>,
       <matplotlib.axes._subplots.AxesSubplot object at 0x7f5f67b2b080>], dtype=object)

The latitude is almost in phase with the phase of the moon, at least at the moment.  

Next job is to add these series to our data frame and then take a look at scikit-learn

In [34]:
print(sdata.index[0])
sdata.index[0].hour + (sdata.index[0].minute / 60.)

15.116666666666667

2015-06-25 15:07:37.174400


In [35]:
sdata.describe()

           altitude  pressure  sealevel_pressure          temp    moon_phase  \
count  24390.000000         0                  0  24390.000000  24390.000000   
mean     -38.176001       NaN                NaN     29.239996     71.811609   
std       17.949836       NaN                NaN      0.816808     25.825346   
min      -69.792586       NaN                NaN     27.200000     12.998277   
25%      -52.066411       NaN                NaN     28.700000     55.731966   
50%      -42.701518       NaN                NaN     29.300000     79.226440   
75%      -28.683335       NaN                NaN     29.800000     94.245584   
max        8.379000       NaN                NaN     31.100000     99.811104   

          moon_glat     moon_glon  
count  24390.000000  24390.000000  
mean       0.030610      3.590249  
std        0.051875      2.041656  
min       -0.078916      0.000126  
25%       -0.007763      1.029049  
50%        0.045898      4.204486  
75%        0.076175      5.

In [36]:
def tide_proxy(df):
    # Create dataframe with index as the index
    series = pandas.Series(index=df.index)
    
    for ix, timestamp in enumerate(df.index):
        hour_min = timestamp.hour + (timestamp.minute / 60.)
	
        hour_min += df.moon_glat[ix]

        series[ix] = hour_min
        
    return series
        


In [16]:
xx = tide_proxy(sdata)

xx.plot()

xx.describe()

count    23229.000000
mean        12.076573
std          6.913170
min         -0.058869
25%          6.086867
50%         12.146825
75%         18.029911
max         24.070675
dtype: float64

In [17]:
# See what we got
moon = moon_orbitals(data.index)
moon.describe()


              phase          glat          glon
count  23229.000000  23229.000000  23229.000000
mean      74.594760      0.035876      3.715151
std       23.181442      0.047355      2.012164
min       19.451408     -0.070132      0.000126
25%       61.817314      0.005469      0.981197
50%       80.961258      0.049373      4.296528
75%       94.750534      0.077198      5.270589
max       99.811104      0.087505      6.283139

In [37]:
sdata['tide'] = tide_proxy(sdata)

In [38]:
fields = ['altitude', 'temp', 'tide']

sdata[fields].plot()

In [39]:
sdata.temp.plot()

In [20]:
with open("../files/moon_weather.csv", 'w') as outfile:
     sdata.to_csv(outfile)